In [76]:
import pandas as pd
import numpy as np
import base64
import requests
import datetime
from sklearn.metrics import mean_squared_error

In [77]:
data = pd.read_csv('test_book.csv')
data.head()

df = data[['userId', 'movieId', 'rating']]
#df.loc[df['userId'] == 1,]
df.head(10)
print(df)
df.shape[0]


    userId  movieId  rating
0        1        2       1
1        1        3       2
2        1        4       2
3        1        5       5
4        1        7       4
5        1        8       3
6        1        9       5
7        2        1       1
8        2        2       5
9        2        3       3
10       2        5       2
11       2        6       3
12       2        7       4
13       2        8       3
14       3        1       1
15       3        2       1
16       3        3       2
17       3        5       2
18       3        6       4
19       3        7       4
20       3        8       5
21       4        1       3
22       4        2       2
23       4        3       2
24       4        4       3
25       4        6       1
26       4        7       3
27       4        8       2
28       5        1       5
29       5        2       1
30       5        3       5
31       5        4       5
32       5        5       4
33       5        6       4
34       5        7 

36

In [78]:
training_data = df.sample(frac=0.8, random_state=25) 
testing_data = df.drop(training_data.index)
testing_data['enjoyed'] = (testing_data['rating'] > 3)

df = training_data #training data will be referred to as df
print(df.shape[0], testing_data.shape[0])

29 7


In [79]:
testing_data.groupby('userId').movieId.agg(list).reset_index()

,userId,movieId
0,1,[7]
1,2,[7]
2,3,"[2, 6]"
3,4,"[3, 7]"
4,5,[2]


In [80]:
user_rated_movies_df = df.groupby('userId').movieId.agg(list).reset_index()
user_rated_movies_df



,userId,movieId
0,1,"[9, 4, 2, 5, 3, 8]"
1,2,"[5, 6, 3, 8, 1, 2]"
2,3,"[5, 1, 7, 3, 8]"
3,4,"[2, 8, 4, 1, 6]"
4,5,"[3, 1, 4, 6, 5, 7, 8]"


In [81]:
df.groupby('movieId').userId.agg(list).reset_index()

,movieId,userId
0,1,"[5, 3, 4, 2]"
1,2,"[4, 1, 2]"
2,3,"[5, 2, 3, 1]"
3,4,"[4, 1, 5]"
4,5,"[2, 3, 5, 1]"
5,6,"[2, 5, 4]"
6,7,"[3, 5]"
7,8,"[4, 2, 1, 3, 5]"
8,9,[1]


In [82]:
users_that_rated_movie = df.groupby('movieId').userId.agg(list).reset_index().set_index('movieId').T.to_dict()
users_that_rated_movie

{1: {'userId': [5, 3, 4, 2]},
 2: {'userId': [4, 1, 2]},
 3: {'userId': [5, 2, 3, 1]},
 4: {'userId': [4, 1, 5]},
 5: {'userId': [2, 3, 5, 1]},
 6: {'userId': [2, 5, 4]},
 7: {'userId': [3, 5]},
 8: {'userId': [4, 2, 1, 3, 5]},
 9: {'userId': [1]}}

In [83]:
# MATRIX FOR RATING OF EACH USER AND MOVIE PAIR

newf = df.pivot(index='userId', columns='movieId')
newf.columns = newf.columns.droplevel(0)

 
matrix = newf.copy()

newf.reset_index(inplace=True)



newf.fillna('NaN', inplace=True)

newf

movieId,userId,1,2,3,4,5,6,7,8,9
0,1,NaN,1.0,2.0,2.0,5.0,NaN,NaN,3.0,5.0
1,2,1.0,5.0,3.0,NaN,2.0,3.0,NaN,3.0,NaN
2,3,1.0,NaN,2.0,NaN,2.0,NaN,4.0,5.0,NaN
3,4,3.0,2.0,NaN,3.0,NaN,1.0,NaN,2.0,NaN
4,5,5.0,NaN,5.0,5.0,4.0,4.0,5.0,2.0,NaN


In [84]:
if 29 not in newf:
    print('not in')
else:
    print('in')

not in


In [85]:
def get_alpha():
    return 0.01

In [86]:
all_users = sorted(list(set(df['userId'])))


def compute_prior(list_of_movies, possible_ratings):
    # Let r_i be the rating of the i-th movie by any user
    # P(r_i = y) where y is any user:
    #           # users | r_u,i = y 
    #           -----------------------
    #           # users | r_u,i =/= NA + # of plausible ratings
    # i.e. P() a movie is rated y = # of usres that rated it y / # of users that rate =/= 'NA' + # of ratings for the movie
    prior_probs = {}
    alpha = get_alpha()

    # prior represents the prob that the item i be rated by ANY user as y

    for movie in list_of_movies:
        for rating in possible_ratings:
            num = len(df.loc[(df['movieId'] == movie) & (df['rating'] == rating) ]) + alpha
            denom = len(users_that_rated_movie[movie]['userId']) + len(possible_ratings) * alpha
            prior_probs[(movie, rating)] = num / denom
        
    for rating in possible_ratings:
        num = alpha
        denom = len(possible_ratings) * alpha
        prior_probs[(np.NaN, rating)] = num/denom

    return prior_probs

list_of_movies = list(set(df['movieId']))
possible_ratings = sorted(list(set(df['rating'])))
prior_probs = compute_prior(list_of_movies, possible_ratings)
prior_probs

{(1, 1): 0.4962962962962963,
 (1, 2): 0.0024691358024691358,
 (1, 3): 0.24938271604938272,
 (1, 4): 0.0024691358024691358,
 (1, 5): 0.24938271604938272,
 (2, 1): 0.3311475409836066,
 (2, 2): 0.3311475409836066,
 (2, 3): 0.0032786885245901644,
 (2, 4): 0.0032786885245901644,
 (2, 5): 0.3311475409836066,
 (3, 1): 0.0024691358024691358,
 (3, 2): 0.4962962962962963,
 (3, 3): 0.24938271604938272,
 (3, 4): 0.0024691358024691358,
 (3, 5): 0.24938271604938272,
 (4, 1): 0.0032786885245901644,
 (4, 2): 0.3311475409836066,
 (4, 3): 0.3311475409836066,
 (4, 4): 0.0032786885245901644,
 (4, 5): 0.3311475409836066,
 (5, 1): 0.0024691358024691358,
 (5, 2): 0.4962962962962963,
 (5, 3): 0.0024691358024691358,
 (5, 4): 0.24938271604938272,
 (5, 5): 0.24938271604938272,
 (6, 1): 0.3311475409836066,
 (6, 2): 0.0032786885245901644,
 (6, 3): 0.3311475409836066,
 (6, 4): 0.3311475409836066,
 (6, 5): 0.0032786885245901644,
 (7, 1): 0.004878048780487806,
 (7, 2): 0.004878048780487806,
 (7, 3): 0.004878048780487

In [87]:
if (2, 3.0) in prior_probs.keys():
    print('yes')

yes


In [88]:
"""print('Enter user id that wants a recommendation:')
test_userId = int(input())
print('Enter movie id for unrated movie: ')
unrated_movieId = int(input())"""

"print('Enter user id that wants a recommendation:')\ntest_userId = int(input())\nprint('Enter movie id for unrated movie: ')\nunrated_movieId = int(input())"

In [89]:
from numpy import NaN


def compute_likelihoood(list_of_movies, possible_ratings, list_of_users, unrated_movieId):
    # need to calculate likelihood
    # P(r_j = k | r_i = y) where j is the j-th movie being rated as 'k', given that the rating of the i-th movie was 'y'
    # ---------------> # users | r_u,j = k AND r_u,i = y
    #                 ----------------------------------------
    #               # users | r_u,j = NA AND r_u,i = y + # of possibilities
    #
    # ======== # of users that rated movie j as 'k' AND movie i rated as 'y' 
    #         ---------------------------------------------------------------
    #       # of users that rated movie j as NA and rated movie i as y + all possibilities
    alpha = get_alpha()
    # posterior_prob = prior_prob P(r_i = y) * P(r_j = k | r_i = y) for every movie j rated by user u
    #user_rated_movies = list(user_rated_movies_df.loc[user_rated_movies_df['userId'] == test_userId, ]['movieId'])[0]
    #print(f"Movies that user {test_userId} rated {user_rated_movies}")

    # likelihood values will only be calculated for movies that are rated by the specific user
    likelihoods = []
  

    df_dict = {}
    for user in list_of_users:
        user_rated_movies = sorted(list(user_rated_movies_df.loc[user_rated_movies_df['userId'] == user, ]['movieId']))[0]
        print(f"Movies that user {user} rated {user_rated_movies}") 
        likelihood_df = pd.DataFrame(columns=possible_ratings)

        for i in range(len(user_rated_movies)):
            rated_movieId = user_rated_movies[i]
            for possible_rating in possible_ratings:
            
                rating = matrix.loc[user, rated_movieId]
                if unrated_movieId not in newf:
                    numerator = alpha
                    denominator = len(possible_ratings) * alpha
                else:
                    numerator = len(newf.loc[(newf[rated_movieId] == rating) & (newf[unrated_movieId] == possible_rating)]) + alpha
                    denominator = len(newf.loc[(newf[rated_movieId] != 'NaN') & (newf[unrated_movieId] == possible_rating)]) + len(possible_ratings) * alpha
        
                likelihood = numerator / denominator
                likelihoods.append(likelihood)
                
            likelihood_df.loc[rated_movieId] = likelihoods
            likelihoods = []
        df_dict[user] = likelihood_df
        #print(likelihood_df)
        
                
    #print(df_dict)

    return df_dict
    

    

list_of_users = list(set(df['userId']))

#df_dict = compute_likelihoood(list_of_movies, possible_ratings, all_users, unrated_movieId=unrated_movieId)

In [90]:
def compute_posterior_probs(df_dict, prior_probs, user, unrated_movieId):
    # first compute product of each column (feature) in likelihood_df
    likelihood_df = df_dict[user]
    col_product = pd.DataFrame(likelihood_df.product()).T

    # multiply each product by corresponding prior (class) prob to get posterior prob
    
    posterior_probs = []
    for i in range(len(col_product.columns)):
        if unrated_movieId not in newf:
            posterior_prob = prior_probs[(np.NaN, col_product.columns[i])] * col_product.iloc[0, i]
        else: 
            posterior_prob = prior_probs[(unrated_movieId, col_product.columns[i])] * col_product.iloc[0, i]
        posterior_probs.append(posterior_prob)

    # make the posterior probs into a dataframe
    posterior_probs_df = pd.DataFrame(posterior_probs).T
    posterior_probs_df.columns = col_product.columns

    #print(posterior_probs_df)

    return posterior_probs_df

#posterior_probs_df = compute_posterior_probs(df_dict=df_dict, prior_probs=prior_probs, user=test_userId)

In [91]:
def arg_max(posterior_probs_df):
    max_rating = np.argmax(posterior_probs_df) + 1
    return max_rating


In [92]:
def evaluation(testing_data):
    # go through the users in testing data, find their predicted ratings for a movie and comapre with actual column
    li = []

    for index, row in testing_data.iterrows():
        test_userId = row['userId']
        unrated_movieId = row['movieId']
        #print(test_userId, unrated_movieId)
        #set unrated_movieId = 1 to check if values are calculated correctly
        df_dict = compute_likelihoood(list_of_movies=list_of_movies, possible_ratings=possible_ratings, list_of_users=list_of_users, unrated_movieId=unrated_movieId)
       # print('here')
        rating = arg_max(compute_posterior_probs(df_dict, prior_probs, test_userId, unrated_movieId))
        li.append(rating)

    testing_data['predicted rating'] = li
    testing_data['predicted enjoyment'] = (testing_data['predicted rating'] > 3)

    
    return testing_data

results = evaluation(testing_data)
results['correct prediction'] = (results['enjoyed'] == results['predicted enjoyment'])
results


Movies that user 1 rated [9, 4, 2, 5, 3, 8]
Movies that user 2 rated [5, 6, 3, 8, 1, 2]
Movies that user 3 rated [5, 1, 7, 3, 8]
Movies that user 4 rated [2, 8, 4, 1, 6]
Movies that user 5 rated [3, 1, 4, 6, 5, 7, 8]
Movies that user 1 rated [9, 4, 2, 5, 3, 8]
Movies that user 2 rated [5, 6, 3, 8, 1, 2]
Movies that user 3 rated [5, 1, 7, 3, 8]
Movies that user 4 rated [2, 8, 4, 1, 6]
Movies that user 5 rated [3, 1, 4, 6, 5, 7, 8]
Movies that user 1 rated [9, 4, 2, 5, 3, 8]
Movies that user 2 rated [5, 6, 3, 8, 1, 2]
Movies that user 3 rated [5, 1, 7, 3, 8]
Movies that user 4 rated [2, 8, 4, 1, 6]
Movies that user 5 rated [3, 1, 4, 6, 5, 7, 8]
Movies that user 1 rated [9, 4, 2, 5, 3, 8]
Movies that user 2 rated [5, 6, 3, 8, 1, 2]
Movies that user 3 rated [5, 1, 7, 3, 8]
Movies that user 4 rated [2, 8, 4, 1, 6]
Movies that user 5 rated [3, 1, 4, 6, 5, 7, 8]
Movies that user 1 rated [9, 4, 2, 5, 3, 8]
Movies that user 2 rated [5, 6, 3, 8, 1, 2]
Movies that user 3 rated [5, 1, 7, 3, 8]
Mov

,userId,movieId,rating,enjoyed,predicted rating,predicted enjoyment,correct prediction
4,1,7,4,True,4,True,True
12,2,7,4,True,4,True,True
15,3,2,1,False,5,True,False
18,3,6,4,True,3,False,False
23,4,3,2,False,1,False,True
26,4,7,3,False,1,False,True
29,5,2,1,False,3,False,True


In [93]:
mse = mean_squared_error(results['enjoyed'].astype(int), results['predicted enjoyment'].astype(int))
mse

0.2857142857142857

### Sources used

In [94]:
'https://www.youtube.com/watch?v=3I8oX3OUL6I'
'https://www.youtube.com/watch?v=lFJbZ6LVxN8'
'https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8787761'

'https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8787761'